In [4]:
import sys
import math
import random
from itertools import permutations
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
# tested with Python 3.7.0 & Gurobi 9.1.0

In [5]:
df_data = pd.read_excel('data')
#clean data
df_clean = df_data[['Agent Coordinates','Quantity','Order Line Total']]
#15 orders were dropped because they did not have delievery location
df_clean = df_clean[df_clean['Agent Coordinates'] !=' ']

#Find unique delivery locations for all orders
df_orders = df_clean.groupby(['Agent Coordinates']).sum()
df_orders.reset_index(inplace=True)


df_orders = df_orders.rename(columns={#'Order ID':'ID',
                  'Quantity':'QTY',
                  'Agent Coordinates':'Coordinates',
                  'Order Line Total':'Total'})

df_orders


,Coordinates,QTY,Total
0,-1.06793800 37.05240120,3.0,455.0
1,-1.06806450 37.05754160,6.0,904.0
2,-1.06806450 37.05754480,10.0,1174.0
3,-1.07090000 37.05840000,1.0,18.0
4,-1.08010000 36.99900000,16.0,730.0
...,...,...,...
640,-1.32600000 36.89160000,30.0,6416.0
641,-1.32660000 36.89300000,8.0,660.0
642,-1.32690000 36.89250000,4.0,212.0
643,-1.32716320 36.89277020,5.0,582.0


In [8]:
  locations_ = []
  for coordinate in df_orders.Coordinates.to_list():
    lst_coordinate = coordinate.split(' ')
    if len(lst_coordinate) == 3:
      locations_.append((float(lst_coordinate[1]),float(lst_coordinate[2])))
    elif len(lst_coordinate) == 2:
      locations_.append((float(lst_coordinate[0]),float(lst_coordinate[1])))
    else:
      continue
locations_
depo = np.array(locations_).mean(axis=0)
locations_.insert(0, (depo[0],depo[1]))

locations_

[(-1.2384171245581388, 36.814971130930246),
 (-1.067938, 37.0524012),
 (-1.0680645, 37.0575416),
 (-1.0680645, 37.0575448),
 (-1.0709, 37.0584),
 (-1.0801, 36.999),
 (-1.0826, 37.0015),
 (-1.0843667, 37.0277517),
 (-1.0905266, 37.0215172),
 (-1.0910397, 37.021047),
 (-1.0915005, 37.0229867),
 (-1.0917, 37.021),
 (-1.0934826, 37.0042426),
 (-1.0935, 37.0048),
 (-1.0947, 37.0033),
 (-1.0953, 37.0242),
 (-1.0969117, 37.007255),
 (-1.0982, 37.0158),
 (-1.0985, 37.0101),
 (-1.0985182, 37.0100239),
 (-1.0993, 37.0106),
 (-1.0998786, 37.008344),
 (-1.1010594, 37.0130843),
 (-1.1018, 37.0096),
 (-1.1019019, 37.0092392),
 (-1.1024, 37.0098),
 (-1.1025, 37.0108),
 (-1.1066, 37.0147),
 (-1.1181708, 37.0079909),
 (-1.1215283, 37.009025),
 (-1.1229, 37.0071),
 (-1.1230587, 37.0074619),
 (-1.1242, 36.9815),
 (-1.1255, 36.9608),
 (-1.1256105, 37.0065263),
 (-1.1257, 36.9638),
 (-1.1269, 36.9657),
 (-1.1299, 36.9699),
 (-1.1334, 36.9838),
 (-1.1347, 36.9767),
 (-1.1388, 36.9734),
 (-1.1392, 36.9872),


In [28]:
# number of locations, including the depot. The index of the depot is 0
n = 646
locations = [*range(n)]

# number of vans
K = 17
vans = [*range(K)]

# Create n random points
points = locations_

# Dictionary of Euclidean distance between each pair of points
# Assume a speed of 60 km/hr, which is 1 km/min. Hence travel time = distance
time = {(i, j):
        math.sqrt(sum((points[i][k]-points[j][k])**2 for k in range(2)))
        for i in locations for j in locations if i != j}

In [29]:
m = gp.Model('lost_luggage_distribution.lp')

# Create variables: 

# x =1, if van  k  visits and goes directly from location  i  to location  j 
x = m.addVars(time.keys(), vans, vtype=GRB.BINARY, name='FromToBy')

# y = 1, if customer i is visited by van k
y = m.addVars(locations, vans, vtype=GRB.BINARY, name='visitBy')

# Number of vans used is a decision variable
z = m.addVars(vans, vtype=GRB.BINARY, name='used')

# Travel time per van
#t = m.addVars(vans, ub=120, name='travelTime')

# Maximum travel time
#s = m.addVar(name='maxTravelTime')

In [30]:
# Van utilization constraint

visitCustomer = m.addConstrs((y[i,k] <= z[k]  for k in vans for i in locations if i > 0), name='visitCustomer' )

# Travel time constraint
# Exclude the time to return to the depot

#travelTime = m.addConstrs((gp.quicksum(time[i,j]*x[i,j,k] for i,j in time.keys() if j > 0) == t[k] for k in vans), 
#                          name='travelTimeConstr' )

# Visit all customers
visitAll = m.addConstrs((y.sum(i,'*') == 1 for i in locations if i > 0), name='visitAll' )


# Depot constraint
depotConstr = m.addConstr(y.sum(0,'*') >= z.sum(), name='depotConstr' )

# Arriving at a customer location constraint
ArriveConstr = m.addConstrs((x.sum('*',j,k) == y[j,k] for j,k in y.keys()), name='ArriveConstr' )

# Leaving a customer location constraint
LeaveConstr = m.addConstrs((x.sum(j,'*',k) == y[j,k] for j,k in y.keys()), name='LeaveConstr' )

breakSymm = m.addConstrs((y.sum('*',k-1) >= y.sum('*',k) for k in vans if k>0), name='breakSymm' )


#maxTravelTime = m.addConstrs((t[k] <= s for k in vans), name='maxTravelTimeConstr')

# Alternately, as a general constraint:
# maxTravelTime = m.addConstr(s == gp.max_(t), name='maxTravelTimeConstr')

In [31]:
m.ModelSense = GRB.MINIMIZE
m.setObjectiveN(z.sum(), 0, priority=1, name="Number of vans")
m.setObjectiveN(s, 1, priority=0, name="Travel time")

GurobiError: Variable not in model

In [ ]:
# Callback - use lazy constraints to eliminate sub-tours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i,j) for i, j, k in model._x.keys()
                                if vals[i, j, k] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n: 
            for k in vans:
                model.cbLazy(gp.quicksum(model._x[i, j, k]
                                         for i, j in permutations(tour, 2))
                             <= len(tour)-1)


# Given a tuplelist of edges, find the shortest subtour not containing depot (0)
def subtour(edges):
    unvisited = list(range(1, n))
    cycle = range(n+1)  # initial length has 1 more city
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            if current != 0:
                unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j == 0 or j in unvisited]
        if 0 not in thiscycle and len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

In [33]:
# Verify model formulation

m.write('lost_luggage_distribution.lp')

# Run optimization engine
m._x = x
m.Params.LazyConstraints = 1
m.setParm('SolutionLimit',1)
m.optimize(subtourelim)

AttributeError: 'gurobipy.Model' object has no attribute 'setParm'

In [27]:
# Print optimal routes
for k in vans:
    route = gp.tuplelist((i,j) for i,j in time.keys() if x[i,j,k].X > 0.5)
    if route:
        i = 0
        print(f"Route for van {k}: {i}", end='')
        while True:
            i = route.select(i, '*')[0][1]
            print(f" -> {i}", end='')
            if i == 0:
                break
        print(f". Travel time: {round(t[k].X,2)} min")

print(f"Max travel time: {round(s.X,2)}")

AttributeError: Unable to retrieve attribute 'X'